## GRIB: writing data into FDB

In [1]:
import os
import pyfdb
import earthkit.data

This example demonstrates how to **write earthkit-data GRIB fields into an FDB**. 

FDB (Fields DataBase) is a domain-specific object store developed at ECMWF for storing, indexing and retrieving GRIB data. For more information on FBD please consult the following pages:

- [FDB](https://fields-database.readthedocs.io/en/latest/)
- [pyfdb](https://pyfdb.readthedocs.io/en/latest/)

#### Setting up the target FDB

In this example we will create an FDB in the current folder using the schema taken from the pyfdb test suite. To do so first we need to ensure the directory exists. Next, we have to specify the configuration via the **FDB5_CONFIG** environment variable.  

In [2]:
fdb_schema = "./default_fdb_schema"
fdb_dir = "./_fdb"
os.makedirs(fdb_dir, exist_ok=True)

c = {"type":"local",
     "engine":"toc",
     "schema":fdb_schema,
     "spaces":[{"handler":"Default",
                "roots":[{"path":fdb_dir}]}]}

os.environ["FDB5_CONFIG"] = str(c)

#### Archiving GRIB into the target FDB

First, we read temperature and wind GRIB data on pressure levels from a file.

In [3]:
earthkit.data.download_example_file("tuv_pl.grib")
ds = earthkit.data.from_source("file", "tuv_pl.grib")
ds.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
2,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
3,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
4,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


Next, we create an fdb object to establish the connection to our target FDB. Finally, the whole fieldlist is written into the FDB field by field.

In [4]:
fdb = pyfdb.FDB()

for f in ds:
    fdb.archive(f.message())

fdb.flush()

#### Testing our FDB

In [5]:
request = {
    'class': 'od',
    'expver': '0001',
    'stream': 'oper',
    'date': '20180801',
    'time': 1200,
    'domain': 'g',
    'type': 'an',
    'levtype': 'pl',
    'levelist': 500,
    'step': 0,
    'param': [131, 132]
}

# read fields from fdb as a stream
for f in earthkit.data.from_source("fdb", request):
    print(f)

GribField(u,500,20180801,1200,0,0)
GribField(v,500,20180801,1200,0,0)
